In [0]:
import pickle
from sklearn.model_selection import train_test_split
import imageio
import scipy.misc
import numpy as np
import os
import keras
from PIL import Image
import cv2



In [0]:
pip install scipy==1.1.0

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
from zipfile import ZipFile
filename ="/content/drive/My Drive/Image_Classification/dataset_image.zip"
with ZipFile (filename, 'r') as zip:
    zip.extractall()

# New Section

In [0]:
# Loading dataset
def load_datasets():
    
    X=[]
    y=[]
    for image_label in label:
        images = os.listdir("/content/dataset_image/"+image_label)
        for image in images:
            img = scipy.misc.imread("/content/dataset_image/"+image_label+"/"+image)
            img = scipy.misc.imresize(img, (224, 224))
            X.append(img)
            y.append(label.index(image_label))


 
    X=np.array(X)
    #X_expanded_dims = np.expand_dims(X, axis=0)
    #X = keras.applications.mobilenet.preprocess_input(X_expanded_dims)
    y=np.array(y)
    return X,y


In [0]:
# Save int2word dict
label = os.listdir("/content/dataset_image")
save_label = open("/content/drive/My Drive/Image_Classification/int_to_word_out.pickle","wb")
pickle.dump(label, save_label)
save_label.close()

In [0]:
label

['FrenchFries', 'Fried_Rice', 'Samosa', 'Pizza', 'Ice_cream', 'Burger']

In [0]:
import numpy
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input
from keras.preprocessing.image import ImageDataGenerator

In [0]:
K.common.set_image_dim_ordering('tf')
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [0]:
def pre_process(X):

    # normalize inputs from 0-255 to 0.0-1.0

    X = X.astype('float32')
    X = X / 255.0

    return X

In [0]:
def one_hot_encode(y):

    # one hot encode outputs
    y = np_utils.to_categorical(y)
    num_classes = y.shape[1]
    return y,num_classes

In [0]:
# load data
X,y=load_datasets()

In [0]:
# pre process
X=pre_process(X)

In [0]:
#one hot encode
y,num_classes=one_hot_encode(y)

In [0]:
#split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

In [0]:
aug = ImageDataGenerator(
                         rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

epochs = 25

In [0]:
base_model = keras.applications.mobilenet.MobileNet(input_tensor= Input(shape=(224, 224,3)),weights='imagenet', include_top=False)    

In [0]:
print(base_model.summary())


In [0]:
print(base_model.summary())


In [0]:
 # Add the fully-connected layers
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu', name='fc2')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax', name='predictions')(x)

# Create your own model
model = Model(inputs=base_model.input, outputs=predictions)


In [0]:
for i,layer in enumerate(model.layers):
  print("{}: {}".format(i,layer))

In [0]:
print(model.summary())

In [0]:
for layer in model.layers[:87]:
    layer.trainable=False
for layer in model.layers[87:]:
    layer.trainable=True

In [0]:
model.summary()

In [0]:
model.compile(Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])


In [0]:
history=model.fit_generator(
    aug.flow(X_train,y_train,batch_size=32),
    validation_data=(X_test,y_test),
    steps_per_epoch=len(X_train)/32,
    epochs=epochs)


In [0]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [0]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [0]:
# serialize model to JSONx
model_json = model.to_json()
with open("/content/drive/My Drive/Image_Classification/models/new_mobilenet_model_face.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Image_Classification/models/new_mobilenet_model_face.h5")
print("Saved model to disk")